# 第5章: 出力フォーマットとClaudeの代弁

- [レッスン](#lesson)
- [演習](#exercises)  
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行してAPI keyを読み込み、`get_completion`ヘルパー関数を設定します。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ],
        system=system
    )
    return message.content[0].text

---

## レッスン

**Claudeは様々な形式で出力をフォーマットできます**。お願いするだけです！

その一つの方法は、XMLタグを使用して回答をその他の余分なテキストから分離することです。XMLタグを使って自分のプロンプトをより明確で解析可能にできることは既に学びました。実際には、**Claudeの出力をより明確で人間に理解しやすくするためにXMLタグを使うよう依頼する**こともできます。

### 例

第2章で前置きを完全に省略することで解決した「詩の前置き問題」を覚えていますか？実際には、**Claudeに詩をXMLタグに入れるよう指示する**ことでも同様の結果を得ることができます。

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

なぜこれをしたいのでしょうか？**XMLタグ内に出力があることで、エンドユーザーがXMLタグ間のコンテンツを抽出する短いプログラムを書くことで、詩だけを確実に取得できる**からです。

このテクニックの拡張版は、**最初のXMLタグを`assistant`ターンに置くこと**です。`assistant`ターンにテキストを置くとき、基本的にClaudeがすでに何かを言ったということ、そしてその地点から続けるべきということを伝えています。このテクニックは「Claudeの代弁」または「Claudeの応答の事前入力」と呼ばれます。

以下では、最初の`<haiku>`XMLタグでこれを行いました。Claudeがどのように私たちが止めた地点から直接続けているかに注目してください。

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claudeは他の出力フォーマットスタイル、特に`JSON`でも優れています。JSON出力を強制したい場合（決定論的ではありませんが、それに近く）、開始ブラケット`{`でClaudeの応答を事前入力することもできます。

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

以下は、**同じプロンプト内の複数の入力変数と出力フォーマット指定、すべてXMLタグを使用して行った**例です。

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### ボーナスレッスン

API経由でClaudeを呼び出す場合、`stop_sequences`パラメータに終了XMLタグを渡すことで、Claudeが希望するタグを出力した後にサンプリングを停止させることができます。これにより、Claudeがすでに求める答えを提供した後の結論的な発言を排除し、お金と最終トークンまでの時間を節約できます。

上記の内容を変更せずにレッスンプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**サンプル実験場**](#example-playground)をご覧ください。

---

## 演習
- [演習 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [演習 5.2 - 二つの俳句](#exercise-52---two-haikus)
- [演習 5.3 - 二つの俳句、二つの動物](#exercise-53---two-haikus-two-animals)

### 演習 5.1 - Steph Curry GOAT
選択を迫られると、ClaudeはMichael Jordanを史上最高のバスケットボール選手として指定します。Claudeに誰か他の人を選ばせることはできるでしょうか？

`PREFILL`変数を変更して、**Claudeに史上最高のバスケットボール選手はStephen Curryであるという詳細な論証をさせて**ください。この演習の焦点なので、`PREFILL`以外は変更しないようにしてください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = ""

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_1_hint)

### 演習 5.2 - 二つの俳句
以下の`PROMPT`をXMLタグを使って修正し、Claudeが一つではなく二つの俳句を動物について書くようにしてください。一つの詩がどこで終わり、もう一つがどこで始まるかが明確である必要があります。

In [ ]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
response = get_completion(PROMPT, prefill=PREFILL)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("cat", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_2_hint)

### 演習 5.3 - 二つの俳句、二つの動物
以下の`PROMPT`を修正して、**Claudeが二つの異なる動物について二つの俳句を作る**ようにしてください。最初の置換には`{ANIMAL1}`を、二番目の置換には`{ANIMAL2}`を代用として使用してください。

In [ ]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags."

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower()) and re.search("cat", text.lower()) and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_3_hint)

### おめでとうございます！

この時点まですべての演習を解決した場合、次の章に進む準備ができています。楽しいプロンプティングを！

---

## サンプル実験場

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのような影響を与えるかを確認するエリアです。

In [ ]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))